In [1]:
from finance_database import Database
import os
import pandas as pd
import numpy as np
import re

In [2]:
db = Database()
con = db.connection
cur = db.cursor

In [3]:
commodities = cur.execute("SELECT name FROM cme_commodities").fetchall()
commodities = [item[0] for item in commodities]
commodities

['Aluminum',
 'Cocoa',
 'Coffee',
 'Copper',
 'Corn',
 'Cotton',
 'Feeder Cattle',
 'Gold',
 'Heating Oil',
 'Lead',
 'Lean Hogs',
 'Live Cattle',
 'Milk',
 'Natural Gas',
 'Oats',
 'Palladium',
 'Platinum',
 'Silver',
 'Soybean',
 'Soybean Meal',
 'Soybean Oil',
 'Sugar',
 'WTI Crude Oil',
 'Wheat',
 'Zinc']

In [4]:
for commodity in commodities:
    print(commodity)
    commodity_id = cur.execute("SELECT id FROM cme_commodities WHERE name = ?", (commodity,)).fetchone()[0]
    df = pd.read_excel(fr"C:\Users\jonas\Desktop\Studium\Master Finance\Masterarbeit\{commodity}.xlsx", index_col= 0)
    df.index = [pd.to_datetime(f"{item.year}-{item.month}-01") for item in df.index]
    df = df.loc[:, [item for item in df.columns if "#ERROR" not in item]]

    old_new_columns = {}
    for column in df.columns:
        date, series = re.findall("([A-Z]{3} [0-9]{4}) - (.+)", column)[0]
        if series == "SETT. PRICE":
            old_new_columns[column] = date + " price"
        elif series == "VOLUME TRADED":
            old_new_columns[column] = date + " volume"
        elif series == "OPEN INTEREST":
            old_new_columns[column] = date + " open_interest"
        else:
            raise ValueError(f"what column is this: {column}")

    df.rename(columns = old_new_columns, inplace=True)

    maturity_dates = set([item.strip(" price"). strip(" volume").strip(" open_interest") for item in df.columns])

    for maturity_date in maturity_dates:
        for index in df.index:
            if maturity_date + " price" not in df.columns:
                continue
            if (len(df.loc[index:, maturity_date + " price"].dropna()) == 0) or (np.isnan(df.loc[index, maturity_date + " price"])):
                continue
            ts = int(index.timestamp())
            maturity_ts = int(pd.to_datetime(maturity_date.replace("2080", "1980")).timestamp())
            price = df.loc[index, maturity_date + " price"]
            if maturity_date + " volume" in df.columns:
                volume = df.loc[index, maturity_date + " volume"]
            else:
                volume = np.NaN
            if maturity_date + " open_interest" in df.columns:
                open_interest = df.loc[index, maturity_date + " open_interest"]
            else:
                open_interest = np.NaN
            cur.execute("INSERT OR IGNORE INTO cme_commodity_prices VALUES (?, ?, ?, ?, ?, ?)", (commodity_id, maturity_ts, ts, price, volume, open_interest))

con.commit()

Aluminum
Cocoa
Coffee
Copper
Corn
Cotton
Feeder Cattle
Gold
Heating Oil
Lead
Lean Hogs
Live Cattle
Milk
Natural Gas
Oats
Palladium
Platinum
Silver
Soybean
Soybean Meal
Soybean Oil
Sugar
WTI Crude Oil
Wheat
Zinc


In [5]:
for commodity in os.listdir("D:\Investing; Statistics; Economics\Investing\Commodities\Data"):
    print(commodity)
    commodity_id = cur.execute("SELECT id FROM cme_commodities WHERE name = ?", (commodity,)).fetchone()[0]
    for file in os.listdir(f"D:\Investing; Statistics; Economics\Investing\Commodities\Data\{commodity}"):
        date = file.strip(f" {commodity} futures data.csv")
        ts = pd.to_datetime(date).timestamp()
        df = pd.read_csv(f"D:\Investing; Statistics; Economics\Investing\Commodities\Data\{commodity}\{file}", index_col=0)
        df.index = [item[:8] for item in df.index]
        if (f"{commodity} Price" in df.columns) and (f"{commodity} Volume" in df.columns):
            df["price"] = df[f"{commodity} Price"]
            df["volume"] = df[f"{commodity} Volume"]
        else:
            df["price"] = df[f"Prior Settle"]
            df["volume"] = df[f"Volume"]
        
        df.index = pd.to_datetime(df.index)
        df.index = [int(date.timestamp()) for date in df.index]
        df["maturity_ts"] = df.index
        df["commodity_id"] = commodity_id
        df["ts"] = ts
        df["open_interest"] = np.NaN
        df = df.loc[:, ["commodity_id", "maturity_ts", "ts", "price", "volume", "open_interest"]]
        cur.executemany("INSERT OR IGNORE INTO cme_commodity_prices VALUES (?, ?, ?, ?, ?, ?)", df.to_numpy())
con.commit()

Aluminum
Cocoa
Coffee
Copper
Corn
Cotton
Feeder Cattle
Gold
Heating Oil
Lead
Lean Hogs
Live Cattle
Milk
Natural Gas
Oats
Palladium
Platinum
Silver
Soybean
Soybean Meal
Soybean Oil
Sugar
Wheat
WTI Crude Oil
Zinc


In [6]:
con.close()